In [0]:
import sys
import os

#go two level up to reach the project root

project_root=os.path.abspath(os.path.join(os.getcwd(), "../.."))

if project_root not in sys.path:
    sys.path.append(project_root)

from modules.utils.date_utils import get_month_start_n_months_ago
from pyspark.sql.functions import date_format

In [0]:
three_months_ago_start = get_month_start_n_months_ago(3)

In [0]:
df= spark.read.table("nyctaxi.02_silver.taxi_trips_enriched").filter(f"tpep_pickup_datetime > '{three_months_ago_start}'")

In [0]:
df= df.withColumn("year_month", date_format(df.tpep_pickup_datetime, "yyyy-MM"))

In [0]:
#write the yellow_trips data in JSON format to the external table  "yellow_trips_export"

df.write.\
    option("path", "abfss://nyctaxi-yellow@nyctaxistorage03022026.dfs.core.windows.net/yellow_trips_export/").\
    mode("append").\
    format("json").\
    partitionBy("vendor", "year_month")\
    .saveAsTable("nyctaxi.04_export.yellow_trips_export")